In [24]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import cv2
import pickle

import dlib
import face_recognition as fr
import matplotlib.pyplot as plt
import re
import glob

import json
from urllib.request import urlopen
import time
import os

In [25]:
def preprocess(name):
    name = re.sub('\(.+\)', '', name)
    name = name.strip()
    
    return name

In [28]:
driver = webdriver.Chrome()
name_list_url = 'https://namu.wiki/w/%EC%97%B0%EC%98%88%EC%9D%B8/%EB%B3%B8%EB%AA%85%20%EB%B0%8F%20%EC%98%88%EB%AA%85'
driver.get(name_list_url)

name_list, error_list = [], []
for i in range(3,18):
    class_ = driver.find_elements_by_class_name('wiki-heading-content')[i]
    raw_name_list = class_.find_elements_by_css_selector('ul.wiki-list li')
    
    for raw_name in raw_name_list:
        split_name = raw_name.text.split('→')
        stage_name = preprocess(split_name[0])
        
        try:
            real_name = re.search('[가-힣]+', split_name[1])[0].strip()
        except Exception as e:
            error_list.append(raw_name.text)
            continue
            
        name_list.append(stage_name + ' ' + real_name)
print(f'{len(name_list)} persons returned')
print(f'{len(error_list)} persons got error')

print('error list : ', error_list)

1602 persons returned
3 persons got error
error list ;  ['동호(前 유키스) - 신동호☆', '마크(NCT) → Mark Lee☆', '유정(라붐) → → 김유정☆']


In [29]:
stage_name = preprocess(error_list[0].split('-')[0])
real_name = re.search('[가-힣]+', error_list[0].split('-')[1])[0]
print(stage_name, real_name)
name_list.append(stage_name + ' ' + real_name)

stage_name = preprocess(error_list[1].split('→')[0])
real_name = re.search('[가-힣A-Za-z ]+', error_list[1].split('→')[1])[0].strip()
print(stage_name, real_name)
name_list.append(stage_name + ' ' + real_name)

stage_name = preprocess(error_list[2].split('→')[0])
real_name = re.search('[가-힣A-Za-z ]+', error_list[2].split('→')[-1])[0].strip()
print(stage_name, real_name)
name_list.append(stage_name + ' ' + real_name)

동호 신동호
마크 Mark Lee
유정 김유정


# Condition 모음

In [30]:
# 썬글라스 착용 여부
def get_brightness_around_eye(image):
    try:
        landmarks = []
        for k, v in fr.face_landmarks(image)[0].items():
            landmarks.extend(v)

        left1, right1 = landmarks[36][0], landmarks[39][0]
        top1, bottom1 = landmarks[37][1], landmarks[41][1]

        left2, right2 = landmarks[42][0], landmarks[45][0]
        top2, bottom2 = landmarks[43][1], landmarks[47][1]
        
        image[top1:bottom1, left1:right1] = np.nan
        image[top2:bottom2, left2:right2] = np.nan
        
        left_glass = image[top1-10:bottom1+10, left1-5:right1+5]
        right_glass = image[top2-10:bottom2+10, left2-5:right2+5]

        luminance_left = np.nanmean(0.2126*left_glass[:,:,0] + 0.7152*left_glass[:,:,1] + 0.0722*left_glass[:,:,2])
        luminance_right = np.nanmean(0.2126*right_glass[:,:,0] + 0.7152*right_glass[:,:,1] + 0.0722*right_glass[:,:,2])
        
        #luminance_left2 = np.nanmean(left_glass[:,:,0] + left_glass[:,:,1] + left_glass[:,:,2])
        #luminance_right2 = np.nanmean(right_glass[:,:,0] + right_glass[:,:,1] + right_glass[:,:,2])
        
        return luminance_left, luminance_right
    
    except:
        return None

# 마스크 착용 여부
def get_brightness_around_mouse(image):
    landmarks = []
    for k, v in fr.face_landmarks(image)[0].items():
        landmarks.extend(v)

    left1, right1 = landmarks[6][0], landmarks[11][0]
    top1, bottom1 = landmarks[2][1], landmarks[6][1]

    mask = image[top1:bottom1, left1:right1]

    mask_luminance = np.nanmean(0.2126*mask[:,:,0] + 0.7152*mask[:,:,1] + 0.0722*mask[:,:,2])
    return mask_luminance

# 얼굴 회전 여부
def ratio_of_face_rotate(image):
    landmarks = []
    for k, v in fr.face_landmarks(image)[0].items():
        landmarks.extend(v)

    left = np.linalg.norm(np.array(landmarks[30]) - np.array(landmarks[2]))
    right = np.linalg.norm(np.array(landmarks[30]) - np.array(landmarks[14]))
    ratio = min(left, right) / max(left, right)
    
    return ratio

In [ ]:
driver = webdriver.Chrome()

search_url = "https://www.google.com/search?q={}&tbm=isch&ved=2ahUKEwjlj4ThuYryAhUTI6YKHfQ_By0Q2-cCegQIABAA&oq=%EA%B9%80%EB%8F%99%EC%9A%B1&gs_lcp=CgNpbWcQA1AAWABglHhoAHAAeACAAQCIAQCSAQCYAQCqAQtnd3Mtd2l6LWltZw&sclient=img&ei=VsIDYeWMMZPGmAX0_5zoAg"

driver.get(search_url.format(name))

In [82]:
for a in driver.find_elements_by_css_selector('div.islrc img.rg_i.Q4LuWd'):
    b = a.get_attribute('src')

In [118]:
def save_image_from_url(image_url):
    with urlopen(image_url) as f:
        with open(f'error/{name}.jpg', 'wb') as file_name:
            img = f.read()
            file_name.write(img)
    img = np.array(Image.open(io.BytesIO(img)))
    #img = fr.load_image_file(file_name)
            
    return img, file_name

In [116]:
driver = webdriver.Chrome()

In [114]:
error_name_list

['적재 정재원',
 '경원 강경원',
 '정욱 정정길',
 'Jun.K 김민준',
 'Crown J 김계훈',
 'Tukutz 김정식',
 '빅죠 벌크',
 '춘자 홍수연',
 'Electronic Boutique 심재현',
 '반효정 반만희',
 '신사동 호랭이 이호양',
 '케빈 백승빈',
 '터틀맨 임성훈',
 '정이랑 정명옥',
 '건우 김건우',
 'PEEJAY 박정철',
 'ND Lee 이창현',
 '탁 이기철',
 '최영원 최영근',
 '계피 임수진',
 '프리마비스타 윤승헌',
 '다빈크 임형빈',
 '김민정 김순애',
 '성유빈 이성익',
 '지현성 박춘현',
 '김지영 김효식',
 '도환 길도환',
 '이숙 이정숙',
 '신사임당 주언규',
 '한재석 한상우',
 '독고진 전성우',
 '황정리 황태수',
 '상아리 장혁준',
 '하명중 하명종',
 '강예원 김지은',
 'ROMEO 박정민',
 '이민우 이동민',
 '최무룡 최한련',
 '안녕하신가영 백가영',
 '랍티미스트 이혁기',
 '거룡 문경석',
 '스페이스 카우보이 박성진',
 '김인문 김인륜',
 'TRIPPY DOG 김정훈',
 '키진 장경진',
 '남철 윤성노',
 '이박사 이용석',
 '정애리 정경복',
 '이레 조서영',
 '이애리수 이음전',
 '밀라논나 장명숙',
 '알로직 장성민',
 'Razer 임유섭',
 '김정애 김정순',
 '지성 박지성',
 '노아 최유빈',
 '박승화 박승진',
 '최연제 김연재',
 '신세영 정정수',
 '샘 김 김건지',
 '개리 강희건',
 '한그루 민한그루',
 '김바다 김정남',
 'XIA 김준수',
 '에이든 권예준',
 '수민 박수민',
 '토리 김지혜',
 '장신영 장신자',
 'HAON 김하온',
 '김구 김원기',
 'JEROME 오성민',
 'MC몽 신동현',
 'wan9u 이완구',
 '미카 이수훈',
 '제프 이재호',
 'LeeZu 이준영',
 '조수미 조수경',
 'M2U 신동휘',
 '김단율 김동현',
 '강석 

In [121]:
error_name_list[48:]

'이레 조서영'

In [122]:
delay=2
error_name = []
#detector_hog = dlib.get_frontal_face_detector() # 얼굴 탐지기 불러오기

for name in error_name_list[48:]:
    search_url = "https://www.google.com/search?q={}&tbm=isch&ved=2ahUKEwjlj4ThuYryAhUTI6YKHfQ_By0Q2-cCegQIABAA&oq=%EA%B9%80%EB%8F%99%EC%9A%B1&gs_lcp=CgNpbWcQA1AAWABglHhoAHAAeACAAQCIAQCSAQCYAQCqAQtnd3Mtd2l6LWltZw&sclient=img&ei=VsIDYeWMMZPGmAX0_5zoAg"
    driver.get(search_url.format(name))
    time.sleep(0.2)
    try:
        driver.find_element_by_tag_name('html').send_keys(Keys.END)
        image_url_list = driver.find_elements_by_css_selector('div.islrc img.rg_i.Q4LuWd')[:50]
    
        is_succeed = 0
        for image_url in image_url_list:
            
            img, file_name = save_image_from_url(image_url.get_attribute('src'))

            try:
#                 image = fr.load_image_file(file_name.name)
#                 if (image.shape[0] < 200) or (image.shape[1] < 200):
#                     print(name, 'Too Small')
#                     continue
                
                locations = fr.face_locations(image)
                if len(locations) != 1:
                    print(name, 'No Face')
                    continue
                    
#                 top, right, bottom, left = locations[0]
#                 face_cropped = image[top:bottom, left:right]
#                 if (face_cropped.shape[0] < 30) or (face_cropped.shape[1] < 30):
#                     print(name, 'Cropped Too Small')
#                     continue
                
                face_embedding = fr.face_encodings(face_cropped)
                if len(face_embedding) != 1:
                    print(name, 'No Embedding')
                    continue
                                  
                ratio = ratio_of_face_rotate(image)
                if ratio < 0.60:
                    print(name, 'Rotated')
                    continue
                    
#                 left_eye, right_eye = get_brightness_around_eye(image)
#                 if (left_eye < 60) & (right_eye < 60):
#                     print(name, 'Maybe Sunglasses')
#                     continue
                
#                 mask = get_brightness_around_mouse(image)
#                 if mask > 220:
#                     print(name, 'Maybe Mask?')
                                
                is_succeed = 1
                print(name, 'Succeed')
                break
                                      
            except Exception as ex:
                print(name, 'failed detection', ex)
                continue
                
        if is_succeed == 0:
            os.remove(file_name.name)
            error_name.append(name)
            print(name, 'collecting Failed')
        
    except Exception as ex:
        print(name, ex)
        error_name.append(name)
        

이레 조서영 Succeed
이애리수 이음전 Succeed
밀라논나 장명숙 Succeed
알로직 장성민 Succeed
Razer 임유섭 Succeed
김정애 김정순 Succeed
지성 박지성 Succeed
노아 최유빈 Succeed
박승화 박승진 Succeed
최연제 김연재 Succeed
신세영 정정수 Succeed
샘 김 김건지 Succeed
개리 강희건 Succeed
한그루 민한그루 Succeed
김바다 김정남 Succeed
XIA 김준수 Succeed
에이든 권예준 Succeed
수민 박수민 Succeed
토리 김지혜 Succeed
장신영 장신자 Succeed
HAON 김하온 Succeed
김구 김원기 Succeed
JEROME 오성민 Succeed
MC몽 신동현 Succeed
wan9u 이완구 Succeed
미카 이수훈 Succeed
제프 이재호 Succeed
LeeZu 이준영 Succeed
조수미 조수경 Succeed
M2U 신동휘 Succeed
김단율 김동현 Succeed
강석 전영근 Succeed
황정자 황창순 Succeed
제이민 신민정 Succeed
Sean2slow 정우섭 Succeed
한복남 한영순 Succeed
김시후 김영준 Succeed
wigen 김현중 Succeed
BLSSD 신재훈 Succeed
한서진 김지현 Succeed
현무 정창화 Succeed
E SENS 강민호 Succeed
이지연 이진영 Succeed
일리닛 최재연 Succeed
태군 김태군 Succeed
뎁 김민경 Succeed
안석환 안진형 Succeed
이동우 김동우 Succeed
나연 한나연 Succeed
Young K 강영현 Succeed
자니 윤 윤종승 Succeed
강하늘 김하늘 Succeed
Jay Lee 이재현 Succeed
연희 김연희 Succeed
승리 이승현 Succeed
조춘 조창성 Succeed
PD대정령 김대현 Succeed
UMC/UW 유승균 [Errno 2] No such file or directory: 'error/UMC/UW 유승균.jpg

# 다운로드 확인

In [108]:
len(name_list)

1599

In [109]:
len(glob.glob('celebrity_google/*.jpg'))

1035

In [111]:
len(error_name)

458